# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# #import csv as data frame
# city_data = pd.read_csv(".\output\city_data.csv")
# city_data.head()

# #Drop cities that are missing values.  & shows stats
# clean_city_data = city_data.dropna()
# clean_city_data.describe()
# clean_city_data.head()

#import csv as data frame for test file

city_data_test = pd.read_csv(".\output\city_data_test.csv")
#city_data_test.head(10)

#convert strings to floats in data frame

# city_data_test["Humidity"] = city_data_test["Humidity"].astype(float)
city_data_test["Humidity"] = city_data_test["Humidity"].astype(float)
city_data_test["Wind Speed"] = city_data_test["Wind Speed"].astype(float)
city_data_test["Max Temp"] = city_data_test["Max Temp"].astype(float)
city_data_test["Cloudiness"] = city_data_test["Cloudiness"].astype(float)
#city_data_test["Date"] = pd.to_datetime(city_data_test["Date"], format = '%Y%m%d')


#Drop cities that are missing value & shows stats for test
clean_city_data_test = city_data_test.dropna()

#clean_city_data_test.describe()

clean_city_data_test.head()


,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,Port Hedland,59.70,30.79,45.00,93.0,4.47,90.0,AU,1.604115e+09
1,New Norfolk,33.28,134.15,68.00,30.0,3.36,20.0,AU,1.604115e+09
2,Mount Gambier,3.07,172.79,82.58,76.0,19.42,63.0,AU,1.604114e+09
3,Port Elizabeth,65.09,12.37,48.20,76.0,13.87,100.0,ZA,1.604115e+09
4,Yulara,21.23,81.63,69.80,64.0,3.36,0.0,AU,1.604115e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create city humidity heat Map of all of Part 1s data 

# Configure gmaps
gmaps.configure(api_key = g_key)

# Store latitude and longitude in locations
locations = clean_city_data_test[["Lat", "Lng"]]

# Convert Humidity to float JIC
rating = clean_city_data_test["Humidity"] 

# Plot Heatmap
fig_13 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig_13.add_layer(heat_layer)

# Display figure
fig_13

# Save a png
#fig_13.figure.savefig('.\output\City_Humidity Heatmap.png')

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
clean_city_data_test.head()
# Reduce amount based on Weather criteria  Humidity <= 75.  Max Temp > 50.  Wind Speed <10.  Cloudiness <30

ideal_city = clean_city_data_test.loc[(clean_city_data_test["Wind Speed"] <10) & 
                                      (clean_city_data_test["Humidity"] < 75) &
                                      (clean_city_data_test["Max Temp"]> 50) &
                                      (clean_city_data_test["Wind Speed"] < 10) &
                                      (clean_city_data_test["Cloudiness"] < 30)
                                     ]


ideal_city.to_csv(r'.\output\ideal_city.csv')
#ideal_city.head(7)

In [5]:
# search for hotels & create new df

#import ideal cities
hotel_df = pd.read_csv('.\output\ideal_city.csv')

# Add Hotel Name 
hotel_df['Hotel Name'] = ""
del hotel_df['Unnamed: 0']
hotel_df.head()


,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date,Hotel Name
0,New Norfolk,33.28,134.15,68.0,30.0,3.36,20.0,AU,1.604115e+09,
1,Yulara,21.23,81.63,69.8,64.0,3.36,0.0,AU,1.604115e+09,
2,Hobart,43.32,145.57,51.8,53.0,9.17,20.0,AU,1.604115e+09,
3,Tairua,17.02,54.09,73.4,38.0,3.36,0.0,NZ,1.604115e+09,
4,Mount Gambier,13.49,106.98,85.6,67.0,7.29,0.0,AU,1.604115e+09,


In [30]:
# find closest hotels

# Define key search parameters
keyword = "hotel"
radius = 5000
type = "hotel"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row[1]
    lng = row[2]
    lat_lon =f"{lat},{lng}"
    city = row[0]
    
    params = {
        "location": lat_lon,
        "keyword": keyword,
        "radius": radius,
        "type": type,
        "key": g_key
    }
    
    response = requests.get(base_url, params=params).json()    
    print(json.dumps(response, indent=4, sort_keys=True))
    
    name_hotel = response["results"]
    print(f'{name_hotel}')
#     try:
#         hotel_name = response["results"]["name"]      
#         hotel_df.loc[index, 'Hotel Name'] = hotel_name
#         print(f'{row[0]} is complete with hotel: {hotel_name}')
#     except Exception:
#         print("Missing field")
#         hotel_df.loc[index, 'Hotel Name'] = "No hotel"
    print('--------')

      
      
        
    
    


TypeError: list indices must be integers or slices, not str

In [15]:
hotel_df.head() 

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date,Hotel Name
0,New Norfolk,33.28,134.15,68.0,30.0,3.36,20.0,AU,1.604115e+09,No hotel
1,Yulara,21.23,81.63,69.8,64.0,3.36,0.0,AU,1.604115e+09,No hotel
2,Hobart,43.32,145.57,51.8,53.0,9.17,20.0,AU,1.604115e+09,No hotel
3,Tairua,17.02,54.09,73.4,38.0,3.36,0.0,NZ,1.604115e+09,No hotel
4,Mount Gambier,13.49,106.98,85.6,67.0,7.29,0.0,AU,1.604115e+09,No hotel


In [ ]:
# find the closest hotel of each type to coordinates

url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

params = {
    "input": "hotel",
    "fields":"name, rating",   
    "locationbias": "circle:5000",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    location = row["City"]
    params['keyword'] = location
    
#     print(f"Retriving Results for Index{index} hotel is {results['name']}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    try:
        hotel_df.loc[index, 'Hotel Name'] = results['name']
    except Exception:
        print("Missing result...skipping.")
    print('-------')
    
    

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# How to Create Markers on a Gmap (Census_States)
# # Create a map using state centroid coordinates to set markers
# marker_locations = census_data[['Latitude', 'Longitude']]

# # Create a marker_layer using the poverty list to fill the info box
# fig = gmaps.figure()
# markers = gmaps.marker_layer(marker_locations,
#     info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
# fig.add_layer(markers)
# fig

In [ ]:
# # Humidity heat map Config gmaps

# params = {
#     "key": g_key,
#     "radius": 50000
# }

# # Use the lat/lan for locations
# for index, row in clean_city_data_test.iterrows():  
#     lat = row['Lat']
#     lng = row['Lng']
#     humidity = row['Humidity']

#     # change location each iteration while leaving original params in place
#     params["location"] = f"{lat},{lng}"
   
#     # Use the search term: "International Airport" and our lat/lng

    
#     # make request and print url
#     name_address = requests.get(base_url, params=params)
    
# #    print the name_address url, avoid doing for public github repos in order to avoid exposing key
# #     print(name_address.url)

# #    # convert to json
#     name_address = name_address.json()
#     print(json.dumps(name_address, indent=4, sort_keys=True))    
    
    
    

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
